In [2]:
conda install -c anaconda beautifulsoup4

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
html = urlopen(url)

In [5]:
soup = BeautifulSoup(html, 'html5lib')
type(soup)

bs4.BeautifulSoup

In [6]:
title = soup.title
print(title)

<title>List of postal codes of Canada: M - Wikipedia</title>


In [7]:
_rows = soup.find_all('tr')
# print(table_rows)

import csv

with open("editors.csv", "wt+", newline="") as f:
    writer = csv.writer(f)
    for row in _rows:
        csv_row = []
        for cell in row.findAll(["td", "th"]):
            csv_row.append(cell.get_text().strip())
        writer.writerow(csv_row)

In [8]:
df = pd.read_csv('editors.csv', error_bad_lines=False)
df.head(10)

b'Skipping line 291: expected 3 fields, saw 31\nSkipping line 292: expected 3 fields, saw 12\nSkipping line 293: expected 3 fields, saw 18\n'


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor
7,M7A,Downtown Toronto,Queen's Park
8,M8A,Not assigned,Not assigned
9,M9A,Queen's Park,Not assigned


In [9]:
for idx, row in df.iterrows():
    if (df.loc[idx,'Neighbourhood'] == "Not assigned") & (df.loc[idx,'Borough'] == "Not assigned"):
        df.drop(idx, axis=0, inplace= True)    
    elif  df.loc[idx,'Neighbourhood'] == "Not assigned":
        df.loc[idx,'Neighbourhood'] = df.loc[idx,'Borough']
#         print(idx)

df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [10]:
df.reset_index(inplace=True)
df['Latitude']=""
df['Longitude']=""
df.head(10)

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,2,M3A,North York,Parkwoods,,
1,3,M4A,North York,Victoria Village,,
2,4,M5A,Downtown Toronto,Harbourfront,,
3,5,M6A,North York,Lawrence Heights,,
4,6,M6A,North York,Lawrence Manor,,
5,7,M7A,Downtown Toronto,Queen's Park,,
6,9,M9A,Queen's Park,Queen's Park,,
7,10,M1B,Scarborough,Rouge,,
8,11,M1B,Scarborough,Malvern,,
9,13,M3B,North York,Don Mills North,,


In [12]:
df.drop('index', axis = 1, inplace = True)
!wget -q -O 'Geospatial.csv' https://cocl.us/Geospatial_data 
print('Data downloaded!')

Data downloaded!


In [13]:
df_csv = pd.read_csv('Geospatial.csv')
df_csv.set_index('Postal Code', inplace=True)
df_csv.head()
# df_csv.dtype

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [15]:
# for idx, row in df.iterrows():
#     psd = df.loc[idx,'Postcode']
#     df.loc[idx, 'Latitude'] = df_csv.loc[psd,'Latitude']       
#     df.loc[idx, 'Longitude'] = df_csv.loc[psd,'Longitude']

df.head()


,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,Harbourfront,43.6543,-79.3606
3,M6A,North York,Lawrence Heights,43.7185,-79.4648
4,M6A,North York,Lawrence Manor,43.7185,-79.4648


In [40]:
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium=0.5.0 --yes 

import folium 
from geopy.geocoders import Nominatim 

address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [41]:
# Create a new Dataframe for values containing Toronto
new_df = df[df['Borough'].str.contains('Toronto') == True]

In [42]:
new_df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,Harbourfront,43.6543,-79.3606
5,M7A,Downtown Toronto,Queen's Park,43.6623,-79.3895
12,M5B,Downtown Toronto,Ryerson,43.6572,-79.3789
13,M5B,Downtown Toronto,Garden District,43.6572,-79.3789
26,M5C,Downtown Toronto,St. James Town,43.6515,-79.3754
35,M4E,East Toronto,The Beaches,43.6764,-79.293
36,M5E,Downtown Toronto,Berczy Park,43.6448,-79.3733
40,M5G,Downtown Toronto,Central Bay Street,43.658,-79.3874
41,M6G,Downtown Toronto,Christie,43.6695,-79.4226
48,M5H,Downtown Toronto,Adelaide,43.6506,-79.3846


In [45]:
# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(new_df['Latitude'], new_df['Longitude'], new_df['Borough'], new_df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto